<a href="https://www.kaggle.com/code/klyushnik/h2o-automl?scriptVersionId=168314246" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
%matplotlib inline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.isotonic import IsotonicRegression
import random
import os
from copy import deepcopy
from functools import partial
from itertools import combinations

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn import preprocessing
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor

import optuna

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool
from h2o.automl import H2OAutoML

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_20/3045106393.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling
h2o:1: DeprecationWarning: Your Python version is 3.10.10. The support for this version will be removed in H2O 3.42.0.1.


/kaggle/input/playground-series-s3e15/sample_submission.csv
/kaggle/input/playground-series-s3e15/data.csv
/kaggle/input/predicting-heat-flux/Data_CHF_Zhao_2020_ATE.csv


# Load data

In [2]:
data = pd.read_csv('/kaggle/input/playground-series-s3e15/data.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s3e15/sample_submission.csv')
original = pd.read_csv('/kaggle/input/predicting-heat-flux/Data_CHF_Zhao_2020_ATE.csv')

In [3]:
data.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [4]:
sample.head()

,id,x_e_out [-]
0,4,0.0
1,7,0.0
2,10,0.0
3,12,0.0
4,23,0.0


In [5]:
train =  pd.concat([data, original])

# Split data

In [6]:
test = train[train['x_e_out [-]'].isnull()]
train = train[train['x_e_out [-]'].notnull()]

In [7]:
train['S_d_e [mm]'] = 1/4*3.14*train['D_e [mm]']**2
train['S_d_h [mm]'] = 1/4*3.14*train['D_h [mm]']**2
test['S_d_e [mm]'] = 1/4*3.14*test['D_e [mm]']**2
test['S_d_h [mm]'] = 1/4*3.14*test['D_h [mm]']**2

train['V_d_e [mm/s]'] = (1000*train['mass_flux [kg/m2-s]'])/(3.14*(train['D_e [mm]']**2/4))
train['V_d_h [mm/s]'] = (1000*train['mass_flux [kg/m2-s]'])/(3.14*(train['D_h [mm]']**2/4))
test['V_d_e [mm/s]'] = (1000*test['mass_flux [kg/m2-s]'])/(3.14*(test['D_e [mm]']**2/4))
test['V_d_h [mm/s]'] = (1000*test['mass_flux [kg/m2-s]'])/(3.14*(test['D_h [mm]']**2/4))

train['Q_D_e'] = train['V_d_e [mm/s]']*train['S_d_e [mm]']
train['Q_D_h'] = train['V_d_h [mm/s]']*train['S_d_h [mm]']
test['Q_D_e'] = test['V_d_e [mm/s]']*test['S_d_e [mm]']
test['Q_D_h'] = test['V_d_h [mm/s]']*test['S_d_h [mm]']

In [8]:
train.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],S_d_e [mm],S_d_h [mm],V_d_e [mm/s],V_d_h [mm/s],Q_D_e,Q_D_h
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6,NaN,91.56240,NaN,4.117411e+04,NaN,3770000.0
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,83.28065,83.28065,72633.919164,7.263392e+04,6049000.0,6049000.0
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5,46.54265,46.54265,43701.851957,4.370185e+04,2034000.0,2034000.0
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0,24.61760,181.36640,149445.924867,2.028490e+04,3679000.0,3679000.0
5,5,NaN,NaN,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6,NaN,2.83385,NaN,1.287295e+06,NaN,3648000.0


In [9]:
test.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],S_d_e [mm],S_d_h [mm],V_d_e [mm/s],V_d_h [mm/s],Q_D_e,Q_D_h
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8,96.71985,96.71985,7092.649544,7092.649544,686000.0,686000.0
7,7,Peskov,tube,18.00,750.0,NaN,10.0,10.0,1650.0,2.2,78.50000,78.50000,9554.140127,9554.140127,750000.0,750000.0
10,10,Thompson,tube,NaN,NaN,NaN,1.9,1.9,152.0,3.2,2.83385,2.83385,NaN,NaN,NaN,NaN
12,12,Thompson,NaN,6.89,7500.0,NaN,NaN,12.8,1930.0,4.8,NaN,128.61440,NaN,58313.843551,NaN,7500000.0
23,23,Beus,annulus,15.51,1355.0,NaN,5.6,15.2,2134.0,2.1,24.61760,181.36640,55041.921227,7471.064100,1355000.0,1355000.0


# Outlier

In [10]:
print('Columns:', train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23094 entries, 0 to 1864
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   23094 non-null  int64  
 1   author               19691 non-null  object 
 2   geometry             19381 non-null  object 
 3   pressure [MPa]       20108 non-null  float64
 4   mass_flux [kg/m2-s]  19867 non-null  float64
 5   x_e_out [-]          23094 non-null  float64
 6   D_e [mm]             19453 non-null  float64
 7   D_h [mm]             19967 non-null  float64
 8   length [mm]          19937 non-null  float64
 9   chf_exp [MW/m2]      23094 non-null  float64
 10  S_d_e [mm]           19453 non-null  float64
 11  S_d_h [mm]           19967 non-null  float64
 12  V_d_e [mm/s]         16761 non-null  float64
 13  V_d_h [mm/s]         17220 non-null  float64
 14  Q_D_e                16761 non-null  float64
 15  Q_D_h                17220 non-null  

In [11]:
def outlier_removal(data , i):
    q1 = np.quantile(data[i] ,0.25)
    q3 = np.quantile(data[i] , 0.75)
    iqr = q3-q1
    lower_tail = q1 - 1.5*iqr
    upper_tail = q3 + 1.5*iqr
    data.drop(data[data[i]> upper_tail].index , inplace = True)
    data.drop(data[data[i]< lower_tail].index ,inplace =True)
    
outlier_list = ['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]',
       'chf_exp [MW/m2]']
#for  i in outlier_list:
    #outlier_removal(train ,i)

# En-code

In [12]:
label_columns = ['author', 'geometry']
for i in label_columns:
    train.loc[:, i] = train[i].fillna('NONE')
    test.loc[:, i] = test[i].fillna('NONE')

In [13]:
le = preprocessing.LabelEncoder()

for i in label_columns:
    train.loc[:, i] = le.fit_transform(train[i])
    test.loc[:, i] = le.fit_transform(test[i])

# Marking

In [14]:
X = train.drop(columns=['x_e_out [-]'])
y = train['x_e_out [-]']
X_total = test.drop(columns=['x_e_out [-]'])

# Aggregate Featrues

In [15]:
class AggFeatureExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, group_col, agg_col, agg_func):
        self.group_col = group_col
        self.group_col_name = ''
        for col in group_col:
            self.group_col_name += col
        self.agg_col = agg_col
        self.agg_func = agg_func
        self.agg_df = None
        self.medians = None
        
    def fit(self, X, y=None):
        group_col = self.group_col
        agg_col = self.agg_col
        agg_func = self.agg_func
        
        self.agg_df = X.groupby(group_col)[agg_col].agg(agg_func)
        self.agg_df.columns = [f'{self.group_col_name}_{agg}_{_agg_col}' for _agg_col in agg_col for agg in agg_func]
        self.medians = X[agg_col].median()
        
        return self
    
    def transform(self, X):
        group_col = self.group_col
        agg_col = self.agg_col
        agg_func = self.agg_func
        agg_df = self.agg_df
        medians = self.medians
        
        X_merged = pd.merge(X, agg_df, left_on=group_col, right_index=True, how='left')
        X_merged.fillna(medians, inplace=True)
        X_agg = X_merged.loc[:, [f'{self.group_col_name}_{agg}_{_agg_col}' for _agg_col in agg_col for agg in agg_func]]
        
        return X_agg
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        X_agg = self.transform(X)
        return X_agg

In [16]:
class Preprocessor:
    def __init__(self, agg_col, agg_func, group_cols):
        self.agg_col = agg_col
        self.agg_func = agg_func
        self.group_cols = group_cols
        
    def preprocess(self, X, X_total):        
        agg_train, agg_test = [], []
        for group_col in self.group_cols:
            agg_extractor = AggFeatureExtractor(group_col=group_col, agg_col=self.agg_col, agg_func=self.agg_func)
            agg_extractor.fit(pd.concat([X, X_total], axis=0))
            agg_train.append(agg_extractor.transform(X))
            agg_test.append(agg_extractor.transform(X_total))
        X = pd.concat([X] + agg_train, axis=1)
        X_total = pd.concat([X_total] + agg_test, axis=1)
        
        X = X.fillna(0)
        X_total = X_total.fillna(0)
                
        return X, X_total

In [17]:
agg_col = [
           'pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]',
       'chf_exp [MW/m2]'
          ]

agg_func = ['mean', 'std', 'median']
group_cols = [['pressure [MPa]'], ['length [mm]'], ['chf_exp [MW/m2]'], 
              ['pressure [MPa]', 'length [mm]'], ['pressure [MPa]', 'chf_exp [MW/m2]'], 
              ['length [mm]', 'chf_exp [MW/m2]'], 
              ['pressure [MPa]', 'length [mm]', 'chf_exp [MW/m2]']]

pp = Preprocessor(agg_col, agg_func, group_cols)
X, X_total = pp.preprocess(X, X_total)

print(f"X shape :{X.shape} , y shape :{y.shape}")
print(f"X_test shape :{X_total.shape}")

X shape :(23094, 141) , y shape :(23094,)
X_test shape :(10415, 141)


# AutoML

In [18]:
startup  = '/home/h2o/bin/aquarium_startup'
shutdown = '/home/h2o/bin/aquarium_stop'

if os.path.exists(startup):
    os.system(startup)
    local_url = 'http://localhost:54321/h2o'
    aquarium = True
else:
    local_url = 'http://localhost:54321'
    aquarium = False
    
import os
import h2o

In [19]:
h2o.init(url=local_url)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppi3hxqhy
  JVM stdout: /tmp/tmppi3hxqhy/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmppi3hxqhy/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,11 months and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_gi5a2m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [20]:
X_y_train_h = h2o.H2OFrame(
    
    pd.concat([X, y], axis =1)
)

X_y_train_h.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:23094
Cols:142

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],S_d_e [mm],S_d_h [mm],V_d_e [mm/s],V_d_h [mm/s],Q_D_e,Q_D_h,pressure [MPa]_mean_pressure [MPa],pressure [MPa]_std_pressure [MPa],pressure [MPa]_median_pressure [MPa],pressure [MPa]_mean_mass_flux [kg/m2-s],pressure [MPa]_std_mass_flux [kg/m2-s],pressure [MPa]_median_mass_flux [kg/m2-s],pressure [MPa]_mean_D_e [mm],pressure [MPa]_std_D_e [mm],pressure [MPa]_median_D_e [mm],pressure [MPa]_mean_D_h [mm],pressure [MPa]_std_D_h [mm],pressure [MPa]_median_D_h [mm],pressure [MPa]_mean_length [mm],pressure [MPa]_std_length [mm],pressure [MPa]_median_length [mm],pressure [MPa]_mean_chf_exp [MW/m2],pressure [MPa]_std_chf_exp [MW/m2],pressure [MPa]_median_chf_exp [MW/m2],length [mm]_mean_pressure [MPa],length [mm]_std_pressure [MPa],length [mm]_median_pressure [MPa],length [mm]_mean_mass_flux [kg/m2-s],length [mm]_std_mass_flux [kg/m2-s],length [mm]_median_mass_flux [kg/m2-s],length [mm]_mean_D_e [mm],length [mm]_std_D_e [mm],length [mm]_median_D_e [mm],length [mm]_mean_D_h [mm],length [mm]_std_D_h [mm],length [mm]_median_D_h [mm],length [mm]_mean_length [mm],length [mm]_std_length [mm],length [mm]_median_length [mm],length [mm]_mean_chf_exp [MW/m2],length [mm]_std_chf_exp [MW/m2],length [mm]_median_chf_exp [MW/m2],chf_exp [MW/m2]_mean_pressure [MPa],chf_exp [MW/m2]_std_pressure [MPa],chf_exp [MW/m2]_median_pressure [MPa],chf_exp [MW/m2]_mean_mass_flux [kg/m2-s],chf_exp [MW/m2]_std_mass_flux [kg/m2-s],chf_exp [MW/m2]_median_mass_flux [kg/m2-s],chf_exp [MW/m2]_mean_D_e [mm],chf_exp [MW/m2]_std_D_e [mm],chf_exp [MW/m2]_median_D_e [mm],chf_exp [MW/m2]_mean_D_h [mm],chf_exp [MW/m2]_std_D_h [mm],chf_exp [MW/m2]_median_D_h [mm],chf_exp [MW/m2]_mean_length [mm],chf_exp [MW/m2]_std_length [mm],chf_exp [MW/m2]_median_length [mm],chf_exp [MW/m2]_mean_chf_exp [MW/m2],chf_exp [MW/m2]_std_chf_exp [MW/m2],chf_exp [MW/m2]_median_chf_exp [MW/m2],pressure [MPa]length [mm]_mean_pressure [MPa],pressure [MPa]length [mm]_std_pressure [MPa],pressure [MPa]length [mm]_median_pressure [MPa],pressure [MPa]length [mm]_mean_mass_flux [kg/m2-s],pressure [MPa]length [mm]_std_mass_flux [kg/m2-s],pressure [MPa]length [mm]_median_mass_flux [kg/m2-s],pressure [MPa]length [mm]_mean_D_e [mm],pressure [MPa]length [mm]_std_D_e [mm],pressure [MPa]length [mm]_median_D_e [mm],pressure [MPa]length [mm]_mean_D_h [mm],pressure [MPa]length [mm]_std_D_h [mm],pressure [MPa]length [mm]_median_D_h [mm],pressure [MPa]length [mm]_mean_length [mm],pressure [MPa]length [mm]_std_length [mm],pressure [MPa]length [mm]_median_length [mm],pressure [MPa]length [mm]_mean_chf_exp [MW/m2],pressure [MPa]length [mm]_std_chf_exp [MW/m2],pressure [MPa]length [mm]_median_chf_exp [MW/m2],pressure [MPa]chf_exp [MW/m2]_mean_pressure [MPa],pressure [MPa]chf_exp [MW/m2]_std_pressure [MPa],pressure [MPa]chf_exp [MW/m2]_median_pressure [MPa],pressure [MPa]chf_exp [MW/m2]_mean_mass_flux [kg/m2-s],pressure [MPa]chf_exp [MW/m2]_std_mass_flux [kg/m2-s],pressure [MPa]chf_exp [MW/m2]_median_mass_flux [kg/m2-s],pressure [MPa]chf_exp [MW/m2]_mean_D_e [mm],pressure [MPa]chf_exp [MW/m2]_std_D_e [mm],pressure [MPa]chf_exp [MW/m2]_median_D_e [mm],pressure [MPa]chf_exp [MW/m2]_mean_D_h [mm],pressure [MPa]chf_exp [MW/m2]_std_D_h [mm],pressure [MPa]chf_exp [MW/m2]_median_D_h [mm],pressure [MPa]chf_exp [MW/m2]_mean_length [mm],pressure [MPa]chf_exp [MW/m2]_std_length [mm],pressure [MPa]chf_exp [MW/m2]_median_length [mm],pressure [MPa]chf_exp [MW/m2]_mean_chf_exp [MW/m2],pressure [MPa]chf_exp [MW/m2]_std_chf_exp [MW/m2],pressure [MPa]chf_exp [MW/m2]_median_chf_exp [MW/m2],length [mm]chf_exp [MW/m2]_mean_pressure [MPa],length [mm]chf_exp [MW/m2]_std_pressure [MPa],length [mm]chf_exp [MW/m2]_median_pressure [MPa],length [mm]chf_exp [MW/m2]_mean_mass_flux [kg/m2-s],length [mm]chf_exp [MW/m2]_std_mass_flux [kg/m2-s],length [mm]chf_exp [MW/m2]_median_mass_flux [kg/m2-s],length [mm]chf_exp [MW/m2]_mean_D_e [mm],length [mm]chf_exp [MW

In [21]:
train_, test_ = X_y_train_h.split_frame([0.8], seed=623)

In [22]:
print("train:%d test:%d" % (train_.nrows, test_.nrows))

train:18582 test:4512


In [23]:
y_reg = "x_e_out [-]"

x_reg = train_.names

In [24]:
aml = H2OAutoML(max_runtime_secs=900, 
                max_runtime_secs_per_model=30, 
                seed=623, 
                project_name='regression', 
                stopping_metric="RMSE", 
                sort_metric="RMSE")

%time 
aml.train(x=x_reg, y=y_reg, training_frame=train_)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11.4 µs
AutoML progress: |
16:01:06.972: _train param, Dropping bad and constant columns: [length [mm]chf_exp [MW/m2]_std_chf_exp [MW/m2], pressure [MPa]length [mm]chf_exp [MW/m2]_std_pressure [MPa], pressure [MPa]chf_exp [MW/m2]_std_pressure [MPa], pressure [MPa]length [mm]chf_exp [MW/m2]_std_length [mm], length [mm]_std_length [mm], pressure [MPa]_std_pressure [MPa], pressure [MPa]length [mm]_std_pressure [MPa], pressure [MPa]length [mm]_std_length [mm], length [mm]chf_exp [MW/m2]_std_length [mm], pressure [MPa]length [mm]chf_exp [MW/m2]_std_chf_exp [MW/m2], chf_exp [MW/m2]_std_chf_exp [MW/m2], pressure [MPa]chf_exp [MW/m2]_std_chf_exp [MW/m2]]

███
16:01:36.680: _train param, Dropping bad and constant columns: [length [mm]chf_exp [MW/m2]_std_chf_exp [MW/m2], pressure [MPa]length [mm]chf_exp [MW/m2]_std_pressure [MPa], pressure [MPa]chf_exp [MW/m2]_std_pressure [MPa], pressure [MPa]length [mm]chf_exp [MW/m2]_std_length [mm], len

key,value
Stacking strategy,cross_validation
Number of base models (used / total),19/39
# GBM base models (used / total),8/15
# XGBoost base models (used / total),10/17
# DeepLearning base models (used / total),1/4
# GLM base models (used / total),0/1
# DRF base models (used / total),0/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [25]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_1_20240322_160106,0.0732937,0.00537196,0.0502778,0.089294,0.00537196
StackedEnsemble_AllModels_4_AutoML_1_20240322_160106,0.073295,0.00537216,0.050281,0.0892915,0.00537216
StackedEnsemble_AllModels_1_AutoML_1_20240322_160106,0.0734763,0.00539876,0.0503623,0.0894686,0.00539876
StackedEnsemble_BestOfFamily_4_AutoML_1_20240322_160106,0.0734766,0.00539881,0.0504671,0.0894851,0.00539881
StackedEnsemble_AllModels_2_AutoML_1_20240322_160106,0.0734864,0.00540024,0.0504076,0.0894549,0.00540024
StackedEnsemble_BestOfFamily_2_AutoML_1_20240322_160106,0.0736706,0.00542736,0.0506189,0.0895842,0.00542736
StackedEnsemble_BestOfFamily_3_AutoML_1_20240322_160106,0.0736865,0.00542971,0.0506666,0.089608,0.00542971
GBM_2_AutoML_1_20240322_160106,0.0738415,0.00545257,0.050698,0.0897933,0.00545257
GBM_grid_1_AutoML_1_20240322_160106_model_5,0.0738581,0.00545502,0.0507481,0.0897534,0.00545502
GBM_grid_1_AutoML_1_20240322_160106_model_3,0.073878,0.00545796,0.0507222,0.0897194,0.00545796


In [26]:
aml.leader

key,value
Stacking strategy,cross_validation
Number of base models (used / total),19/39
# GBM base models (used / total),8/15
# XGBoost base models (used / total),10/17
# DeepLearning base models (used / total),1/4
# GLM base models (used / total),0/1
# DRF base models (used / total),0/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [27]:
preds = aml.predict(test_)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [28]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_BestOfFamily" in mid][0])
# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])
metalearner.coef()

/opt/conda/lib/python3.10/site-packages/h2o/estimators/stackedensemble.py:926: H2ODeprecationWarning: The usage of stacked_ensemble.metalearner()['name'] will be deprecated. Metalearner now returns the metalearner object. If you need to get the 'name' please use stacked_ensemble.metalearner().model_id
  warnings.warn(


{'Intercept': 0.00011227430532005917,
 'GBM_2_AutoML_1_20240322_160106': 0.5778854860028712,
 'XGBoost_grid_1_AutoML_1_20240322_160106_model_11': 0.351102014829217,
 'DeepLearning_grid_1_AutoML_1_20240322_160106_model_2': 0.07166516100654655,
 'GLM_1_AutoML_1_20240322_160106': 0.0,
 'DRF_1_AutoML_1_20240322_160106': 0.018873437367306733,
 'XRT_1_AutoML_1_20240322_160106': 0.005211320712829451}

In [29]:
metalearner

,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 1.072E-5 )",6,5,1,levelone_training_StackedEnsemble_BestOfFamily_4_AutoML_1_20240322_160106
,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.0504626,0.0007895,0.0502238,0.0495974,0.0517491,0.0502819,0.0504607
mean_residual_deviance,0.0053987,0.0002559,0.0051484,0.0052564,0.0055336,0.0052748,0.0057802
mse,0.0053987,0.0002559,0.0051484,0.0052564,0.0055336,0.0052748,0.0057802
null_deviance,39.16138,1.8194162,36.427887,38.770885,40.36432,39.043858,41.199947
r2,0.4873788,0.0095125,0.4770304,0.495979,0.4823751,0.4989115,0.482598
residual_deviance,20.064138,0.9768356,19.038889,19.522408,20.889355,19.564348,21.305689
rmse,0.0734595,0.0017306,0.0717525,0.0725013,0.0743882,0.0726280,0.0760274
rmsle,0.0891288,0.0093024,0.0877448,0.0856828,0.0865411,0.0806283,0.1050472


In [30]:
pred = aml.leader.predict(test_)
pred = pred.cbind(test_['x_e_out [-]'])

pred.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,x_e_out [-]
0.0519115,0.1092
-0.124219,-0.0638
0.038425,0.0196
0.110434,0.0406
0.107804,0.0409
0.0622809,0.0763
0.0136744,0.0063
0.0657459,0.1261
0.128467,0.0845
0.0823292,0.033


# Predict

In [31]:
X_y_test_h = h2o.H2OFrame(X_total)
predict_ = aml.leader.predict(X_y_test_h)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [32]:
predict_h2o = h2o.as_list(predict_)

In [33]:
predict_h2o

,predict
0,0.013879
1,-0.109091
2,-0.047486
3,0.001818
4,0.044233
...,...
10410,0.069320
10411,-0.056535
10412,0.040005
10413,-0.057779


In [34]:
h2o.cluster().shutdown()

H2O session _sid_be05 closed.


# Submit

In [35]:
sample['x_e_out [-]'] = predict_h2o.predict
sample

,id,x_e_out [-]
0,4,0.013879
1,7,-0.109091
2,10,-0.047486
3,12,0.001818
4,23,0.044233
...,...,...
10410,31633,0.069320
10411,31634,-0.056535
10412,31637,0.040005
10413,31640,-0.057779


In [36]:
sample.to_csv('submission.csv', index=False)